# Trial Code

In [2]:
from itertools import groupby
from pycocotools import mask as mutils
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import pickle
import cv2
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [6]:
exp_name = "v3"
conf_name = "mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco"
cell_mask_dir = '../input/hpa-mask/hpa_cell_mask'    
ROOT = '../../Protein_Atlas/'
train_or_test = 'train'
img_dir = f'work/mmdet_{exp_name}_{train_or_test}'
!mkdir -p {img_dir}
df = pd.read_csv(os.path.join(ROOT, 'train.csv'))

# this script takes more than 9hours for full data.
debug = True
if debug:
    df = df[:4]

In [7]:
print(df)

                                     ID  Label
0  5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0  8|5|0
1  5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0   14|0
2  60b57878-bb99-11e8-b2b9-ac1f6b6435d0    6|1
3  5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0  16|10


## Helper Functions

In [8]:
# convert segmentation mask image to run length encoding
MAX_GREEN = 64 # filter out dark green cells
def get_rles_from_mask(image_id, class_id):
    mask = np.load(f'{cell_mask_dir}/{image_id}.npz')['arr_0']
    if class_id != '18':
        green_img = read_img(image_id, 'green')
    rle_list = []
    mask_ids = np.unique(mask)
    for val in mask_ids:
        if val == 0:
            continue
        binary_mask = np.where(mask == val, 1, 0).astype(bool)
        if class_id != '18':
            masked_img = green_img * binary_mask
            #print(val, green_img.max(),masked_img.max())
            if masked_img.max() < MAX_GREEN:
                continue
        rle = coco_rle_encode(binary_mask)
        rle_list.append(rle)
    return rle_list, mask.shape[0], mask.shape[1]


In [6]:
def coco_rle_encode(mask):
    rle = {'counts': [], 'size': list(mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [7]:
# mmdet custom dataset generator
def mk_mmdet_custom_data(image_id, class_id):
    rles, height, width = get_rles_from_mask(image_id, class_id)
    if len(rles) == 0:
        return {
            'filename': image_id+'.jpg',
            'width': width,
            'height': height,
            'ann': {}
        }
    rles = mutils.frPyObjects(rles, height, width)
    bboxes = mutils.toBbox(rles)
    bboxes[:, 2] += bboxes[:, 0]
    bboxes[:, 3] += bboxes[:, 1]
    return {
        'filename': image_id+'.jpg',
        'width': width,
        'height': height,
        'ann':
            {
                'bboxes': np.array(bboxes, dtype=np.float32),
                'labels': np.zeros(len(bboxes)), # dummy data.(will be replaced later)
                'masks': rles
            }
    }

In [8]:
# print utility from public notebook
def print_masked_img(image_id, mask):
    img = load_RGBY_image(image_id, train_or_test)
    
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.6)
    plt.title('Image + Mask')
    plt.axis('off')
    plt.show()

In [9]:
# image loader, using rgb only here
def load_RGBY_image(image_id, train_or_test='train', image_size=None):
    red = read_img(image_id, "red", train_or_test, image_size)
    green = read_img(image_id, "green", train_or_test, image_size)
    blue = read_img(image_id, "blue", train_or_test, image_size)
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

In [ ]:
def read_img(image_id, color, train_or_test='train', image_size=None):
    filename = f'{ROOT}/{train_or_test}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.max() > 255:
        img_max = img.max()
        img = (img/255).astype('uint8')
    return img

In [ ]:
# make annotation helper called multi processes
def mk_ann(idx):
    image_id = df.iloc[idx].ID
    class_id = df.iloc[idx].Label
    anno = mk_mmdet_custom_data(image_id, class_id)
    img = load_RGBY_image(image_id, train_or_test)
    cv2.imwrite(f'{img_dir}/{image_id}.jpg', img)
    return anno, idx, image_id

## Generate files for mmdetection

MMDetection is an open source object detection toolbox based on PyTorch. It is a part of the OpenMMLab project developed by Multimedia Laboratory, CUHK.

In [9]:
out_image_dir = f'/home/chad/GitHub/MMDetection/mmdetection/work/mmdet_{exp_name}_{train_or_test}/'
!mkdir -p {out_image_dir}

annos = []
for idx in tqdm(range(len(df))):
    image_id = df.iloc[idx].ID
    img = load_RGBY_image(image_id, train_or_test)
    
    cv2.imwrite(f'{out_image_dir}/{image_id}.jpg', img)
    ann = {
        'filename': image_id+'.jpg',
        'width': img.shape[1],
        'height': img.shape[0],
        'ann': {
            'bboxes': None,
            'labels': None,
            'masks': None
        }
    }
    annos.append(ann)
    
with open(f'/home/chad/GitHub/MMDetection/mmdetection/work/mmdet_{exp_name}_tst.pkl', 'wb') as f:
    pickle.dump(annos, f)

100%|██████████| 3/3 [00:00<00:00,  3.98it/s]


## Inference

In [14]:
import os 
os.getcwd()

'/home/chad/GitHub/MMDetection/mmdetection'

In [34]:
#exp_name = "v4"
#conf_name = "mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco"
#model_name = 'mask_rcnn_resnest101_v5_ep9'

config = f'configs/hpa_{exp_name}/{conf_name}.py'
print(config)

configs/hpa_v4/mask_rcnn_s101_fpn_syncbn-backbone+head_mstrain_1x_coco.py


In [35]:
model_file = f'../input/hpa-models/{model_name}.pth'
print(model_file)

../input/hpa-models/mask_rcnn_resnest101_v5_ep9.pth


In [31]:
result_pkl = f'../work/{model_name}.pkl'


In [32]:
additional_conf = '--cfg-options'
additional_conf += ' test_cfg.rcnn.score_thr=0.001'


In [33]:
cmd = f'python tools/test.py {config} {model_file} --out {result_pkl} {additional_conf}'
!cd ../mmdetection; {cmd}


Traceback (most recent call last):
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/utils/registry.py", line 179, in build_from_cfg
    return obj_cls(**args)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmdet/datasets/custom.py", line 87, in __init__
    self.data_infos = self.load_annotations(self.ann_file)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmdet/datasets/custom.py", line 112, in load_annotations
    return mmcv.load(ann_file)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/io.py", line 41, in load
    obj = handler.load_from_path(file, **kwargs)
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/handlers/pickle_handler.py", line 13, in load_from_path
    return super(PickleHandler, self).load_from_path(
  File "/home/chad/anaconda3/lib/python3.8/site-packages/mmcv/fileio/handlers/base.py", line 20, in load_from_path
    with open(filepath, mode) as f:
FileNotFoundError: [Errno 2] No such file or d

In [ ]:
result = pickle.load(open('../mmdetection/'+result_pkl, 'rb'))

## Result Check

In [23]:
for ii in range(3):
    image_id = annos[ii]['filename'].replace('.jpg','').replace('.png','')
    for class_id in range(19):
        #print(ii,class_id,len(result[ii][0][class_id]), len(result[ii][1][class_id]))
        bbs = result[ii][0][class_id]
        sgs = result[ii][1][class_id]
        for bb, sg in zip(bbs,sgs):
            box = bb[:4]
            cnf = bb[4]
            h = sg['size'][0]
            w = sg['size'][0]
            if cnf > 0.3:
                print(f'class_id:{class_id}, image_id:{image_id}, confidence:{cnf}')
                mask = mutils.decode(sg).astype(bool)
                print_masked_img(image_id, mask)

KeyError: 0

In [22]:
print(annos)

[{'filename': '0040581b-f1f2-4fbe-b043-b6bfea5404bb.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}, {'filename': '004a270d-34a2-4d60-bbe4-365fca868193.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}, {'filename': '00537262-883c-4b37-a3a1-a4931b6faea5.jpg', 'width': 2048, 'height': 2048, 'ann': {'bboxes': None, 'labels': None, 'masks': None}}]
